In [6]:
# 建立整體的ui介面，變成一個問答機器人
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

/Users/mangtinglee/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
%pip install -U langchain-community pypdf

In [69]:
'''
目的：建立langchain + openai 的基礎環境
'''
import os # 作業系統相關功能（讀取環境變數）
from openai import OpenAI # openai api 客戶端
from dotenv import load_dotenv, find_dotenv # dotenv 是專門用來讀取.env套件的套件，並接上環境
_ = load_dotenv(find_dotenv()) # 讀取.env檔案
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY']
)
# openai.api_key  = os.environ['OPENAI_API_KEY'] 舊版
print("done")

done


# 第一步測試：載入

In [3]:
from dotenv import load_dotenv
import os

# 清除舊的環境變數
if 'OPENAI_API_KEY' in os.environ:
    del os.environ['OPENAI_API_KEY']

# 重新載入
load_dotenv()

api_key = os.environ.get('OPENAI_API_KEY')
if api_key:
    print("✅ Success! API key loaded")
    print(f"Key starts with: {api_key[:15]}...")
    print(f"Key length: {len(api_key)} characters")
else:
    print("❌ Still not working")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10


✅ Success! API key loaded
Key starts with: sk-proj-FWUUter...
Key length: 164 characters


In [4]:
# 檢查資料夾是否存在
import os
folder_path = "/Users/mangtinglee/Desktop/2025_gap_careerpath/RAG_LLM/pdfs"

print(f"Folder exists: {os.path.exists(folder_path)}")
# 看看有哪些PDF檔案
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
print(f"Found {len(pdf_files)} PDF files:")
for pdf in pdf_files[:5]:  # 顯示前5個檔名
    print(f"  - {pdf}")

Folder exists: True
Found 23 PDF files:
  - 2023_LLM limitation_Large Language Models Sensitivity to The Order of Options in Multiple-Choice Questions.pdf
  - 2025_LLM limitation_The Order Effect- Investigating Prompt Sensitivity to Input Order in LLMs.pdf
  - 2024_RAG_Evaluation of Retrieval-Augmented Generation- A Survey.pdf
  - 2020_scaling laws_Scaling Laws for Neural Language Models.pdf
  - 2022_LLM limitation_Robustness of Learning from Task Instructions.pdf


In [15]:
# 測試1.載入一篇
test_file = os.path.join(folder_path, pdf_files[0])
loader = PyPDFLoader(test_file)
documents = loader.load()
print(f"Test file loaded: {len(documents)} pages")
print(f"內容：{documents[3].page_content[:500]}...")
print(f"metadata：{documents[0].metadata}")

Test file loaded: 11 pages
內容：Figure 2: Order sensitivity in few-shot setting:The error bars represent the range of minimum and maximum
accuracy achievable in each task through oracle reordering. Our observations are as follows: (1) The sensitivity
gap consistently remains substantial even with the addition of more demonstrations in the few-shot setting. (2) As
performances improve, the sensitivity gap shrinks. (3) Adding more demonstrations does not necessarily result in a
reduction of the sensitivity gap.
4 Why Do LLMs Sho...
metadata：{'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-23T01:15:54+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-23T01:15:54+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/mangtinglee/Desktop/2025_gap_careerpath/RAG_LLM/pdfs/2023_LLM limitation_Large Language Models Sensit

# 第二步測試：分割檔案

In [72]:
# 測試2. 分割檔案
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=150,
    separators=[ "\n\n", ". ", "\n", "(?<=\. )", " ", ""]
    )    
docs = text_splitter.split_documents(documents)
print(len(docs)) # chunks
print(len(documents)) # test 11 pages

54
11


In [ ]:
print(docs[0].page_content)

```python
separators=[
    "\n\n",        # 段落分隔（最優先）
    ". ",          # 句號
    "\n",          # 行分隔
    "(?<=\. )",    # 句號後分隔（用正則表達式）
    " ",           # 空格分隔
    ""             # 字元分隔（最後手段）
]
```

##### 問題：如何決定塊的大小和重疊度？
- 兩種指標：
    - 統計指標：
        - 平均長度：接近設定的 `chunk _size`，代表接近目標 
        - max：評估是否超過限制
        - min：評估最小塊是否能接受
        - std：chunk的大小範圍大概落在 mean ± std 之間，有點大的話大概長度不太一致，<200 (長度較一致)
        - 一個最短的chunk：通常越少越好，<5%  

    - 語意完整性（更重要）：
        - 一個塊是否包含完整概念？
        - 重要術語是否有被切斷？
        - 評估方法：人工檢視（看完文字是否能理解）、問答測試法（先測試到最後，靠ai檢索是否能從準確回應答案）
- 結論：現在看看不準，要等run過一遍再來看llm的檢索能力
- trade-off解決：語意完整性 > 統計指標

In [73]:
# 檢查chunk是否合理
def analyze_chunks(docs):
    lengths = [len(d.page_content) for d in docs]
    
    print(f"Total chunks: {len(docs)}")
    print(f"Average length: {sum(lengths)/len(lengths):.0f}")
    print(f"Min length: {min(lengths)}")  
    print(f"Max length: {max(lengths)}")
    print(f"Length std dev: {(sum((x-sum(lengths)/len(lengths))**2 for x in lengths)/len(lengths))**0.5:.0f}")
    
    # 檢查是否有太短的chunk（可能是切割錯誤）
    short_chunks = [i for i, l in enumerate(lengths) if l < 200]
    if short_chunks:
        print(f"Warning: {len(short_chunks)} chunks are very short")
        
analyze_chunks(docs)

Total chunks: 54
Average length: 784
Min length: 125
Max length: 998
Length std dev: 215


# 第三步測試：embedding 並放入資料庫

In [ ]:
# 安裝新套件
%pip install -U langchain-openai
%pip install chromadb

In [81]:
# 建立資料庫路徑，已有路徑則可忽略
import os

# 建立資料夾
os.makedirs('./chroma_db', exist_ok=True)
print("finish！")

# 檢查是否成功
print(f"資料夾存在嗎？{os.path.exists('./chroma_db')}")

finish！
資料夾存在嗎？True


In [84]:
# define embedding
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


In [87]:
# 注意，需要先在自己的環境中建立資料庫路徑
persist_directory = './chroma_db' # 指定資料庫路徑
!rm -rf ./chroma_db  # remove old database files if any

In [88]:
# 建立新的向量資料庫，並將文件放進去
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [89]:
# 檢查剛剛的向量行數是否與塊數相等
print(vectordb._collection.count())


54


In [92]:
question = "can you explain the abstract in this article?"
ans_docs = vectordb.similarity_search(question,k=3)
print(len(ans_docs))
print(ans_docs[1].page_content)

3
. Through these investigations, we
contribute to a deeper understanding of how the
order of options affects LLMs’ decision-making in
multiple-choice questions (MCQ) and offer practi-
cal solutions, which go beyond simple bootstrap-
ping, to increase their robustness and accuracy in
such scenarios.
2 Background and Experimental Details
This paper focuses on the task of multiple-choice
question answering. In multiple-choice questions,
the objective is to identify the correct answer to a
given question from a set of possible options (an il-
lustration is presented in Figure 1). To address this
task using in-context learning models, we present
a prompt in the following format: “Choose the
answer to the question only from A, B, C, D,
and E choices. Question: {question}. Choices:
{options}. Answer:” to the models


In [91]:
# 手動儲存剛剛建立的資料庫
vectordb.persist() 

/var/folders/_x/xcmm2w3d1hld6_nc6t0xyrch0000gn/T/ipykernel_75980/2355425439.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [11]:
# rag邏輯
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 


In [ ]:
# 作用：管理對話流程、連接rag和操作介面
    # 例如：記住對話歷史、呼叫rag回答問題、整理答案給使用者看、提供輔助功能
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "docs/cs229_lectures/MachineLearning-Lecture01.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [ ]:
# 網頁介面設計
    # 單純提供視覺介面讓大家可以操作
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard